In [2]:
import cv2,time,pandas
from datetime import datetime


In [3]:
video=cv2.VideoCapture(0)
first_frame=None

df=pandas.DataFrame(columns=["Time in","Time out"])

status_list=[None,None]
time_in_out=[]
while True:
    check,frame=video.read()
    
    status=0
    
    grey_img=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    grey_img=cv2.GaussianBlur(grey_img,(21,21),0)
    
    if first_frame is None:
        first_frame= grey_img
        continue
    
    difference_frame=cv2.absdiff(first_frame,grey_img)

    threshold_frame=cv2.threshold(difference_frame,30,255,cv2.THRESH_BINARY)[1]
    threshold_frame=cv2.dilate(threshold_frame,None,iterations=2)
    
    cont,_=cv2.findContours(threshold_frame.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for contours in cont:
        if cv2.contourArea(contours) <10000:
            continue
        
        status=1
        
        (x,y,w,h)=cv2.boundingRect(contours)
        cv2.rectangle(frame,  (x,y), (x+w,y+h),  (0,255,0),3)
    
    status_list.append(status)
    
    if status_list[-1]==1 and status_list[-2]==0:
        time_in_out.append(datetime.now())
        
    if status_list[-1]==0 and status_list[-2]==1:
        time_in_out.append(datetime.now())
    
    cv2.imshow("grey frame",grey_img)
    cv2.imshow("difference frame",difference_frame)
    cv2.imshow("threshold frame",threshold_frame)
    cv2.imshow("colour frame",frame)
    
    key=cv2.waitKey(1)
    
    if key==ord("q"):
        if status==1:
            time_in_out.append(datetime.now())
        break
print(time_in_out)    

for i in range(0,len(time_in_out),2):
    df=df.append({"Entry time": time_in_out[i], "Exit time": time_in_out[i+1]},ignore_index=True)
    

df.to_csv("Timing for Motion Detector.csv")
    

video.release()
cv2.destroyAllWindows()

[datetime.datetime(2020, 7, 15, 0, 49, 48, 306543), datetime.datetime(2020, 7, 15, 0, 49, 49, 621980), datetime.datetime(2020, 7, 15, 0, 49, 51, 720891), datetime.datetime(2020, 7, 15, 0, 49, 53, 306015), datetime.datetime(2020, 7, 15, 0, 49, 55, 974553), datetime.datetime(2020, 7, 15, 0, 49, 57, 797771)]


In [8]:
df

,Time in,Time out,Entry time,Exit time
0,NaN,NaN,2020-04-18 11:48:40.290044,2020-04-18 11:48:40.934170
1,NaN,NaN,2020-04-18 11:48:44.062875,2020-04-18 11:48:45.019595
